## Part 1

### data

In [1]:
!pip install huggingface_hub


In [2]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="DaoHuyen/XLA",
    repo_type="model",
    local_dir="data_downloaded"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

weights/65_The_best_weights_raf_new.weig(…):   0%|          | 0.00/167M [00:00<?, ?B/s]

weights/65_74_RAF_best_model.weights.h5:   0%|          | 0.00/76.0M [00:00<?, ?B/s]

weights/RAF_best_model.weights.h5:   0%|          | 0.00/101M [00:00<?, ?B/s]

...

data/rafdb_dataset_3.npz:   0%|          | 0.00/2.03G [00:00<?, ?B/s]

data/ck2.npz:   0%|          | 0.00/63.0M [00:00<?, ?B/s]

data/ck.npz:   0%|          | 0.00/63.0M [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

'/content/data_downloaded'

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Model
import h5py

# Load
data = np.load("/content/data_downloaded/data/ck2.npz")

x = data["x"]
y = data["y"]


In [4]:
import numpy as np

# tạo permutation ngẫu nhiên
idx = np.random.permutation(len(x))

# shuffle x và y theo cùng thứ tự
x = x[idx]
y = y[idx]


In [5]:
print(x.shape, y.shape)

(920, 224, 224, 3) (920,)


### model

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Add, MaxPooling2D, AveragePooling2D, Activation, Dense, PReLU, Layer
from tensorflow.keras.layers import Input, BatchNormalization, GlobalAveragePooling2D, Concatenate, Cropping2D, Multiply, Lambda, Flatten, Reshape
from tensorflow.keras.activations import relu, softmax, sigmoid, tanh
from tensorflow.keras import initializers
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

import tensorflow.keras.backend as K

class Patches(Layer):
    def __init__(self, patch_size, **kwargs):
        super(Patches, self).__init__(**kwargs)
        self.patch_size = patch_size

    def call(self, inputs, **kwargs):
        patch = tf.concat((tf.split(inputs,num_or_size_splits=7,axis=1)), axis=-1)
        patch = tf.concat((tf.split(patch, num_or_size_splits=7, axis=2)), axis=-1)
        return patch

    def get_config(self):
        config = {'patch_size':self.patch_size}
        base_config = super(Patches, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def Global_Net(input, eps):
    x_g_1 = Conv2D(filters=64, kernel_size=(7, 7), strides=(1, 1), padding='same', activation=relu)(input)
    x_g_1 = BatchNormalization(axis=-1, epsilon=eps)(x_g_1)
    x_g_1 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x_g_1)
    x_g_1 = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation=relu)(x_g_1)
    x_g_1 = BatchNormalization(axis=-1, epsilon=eps)(x_g_1)
    x_g_1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x_g_1)

    x_g_2 = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation=relu)(x_g_1)
    x_g_2 = BatchNormalization(axis=-1, epsilon=eps)(x_g_2)
    x_g_2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x_g_2)

    x_g_3 = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation=relu)(x_g_2)
    x_g_3 = BatchNormalization(axis=-1, epsilon=eps)(x_g_3)
    x_g_3 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x_g_3)

    x_g_4 = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding='same', activation=relu)(x_g_3)
    x_g_4 = BatchNormalization(axis=-1, epsilon=eps)(x_g_4)
    x_g_4 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x_g_4)

    return x_g_4

def Local_Net(input, eps):
    x_l_1 = Conv2D(filters=128, kernel_size=(7, 7), strides=(1, 1), padding='same', activation=relu)(input)
    x_l_1 = BatchNormalization(axis=-1, epsilon=eps)(x_l_1)
    x_l_1 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x_l_1)
    x_l_1 = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation=relu)(x_l_1)
    x_l_1 = BatchNormalization(axis=-1, epsilon=eps)(x_l_1)
    x_l_1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x_l_1)

    x_l_2 = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation=relu)(x_l_1)
    x_l_2 = BatchNormalization(axis=-1, epsilon=eps)(x_l_2)
    x_l_2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x_l_2)

    x_l_3 = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding='same', activation=relu)(x_l_2)
    x_l_3 = BatchNormalization(axis=-1, epsilon=eps)(x_l_3)
    x_l_3 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x_l_3)

    x_l_4 = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding='same', activation=relu)(x_l_3)
    x_l_4 = BatchNormalization(axis=-1, epsilon=eps)(x_l_4)
    x_l_4 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x_l_4)

    return x_l_4

def Enhance_Net(eps = 1.1e-5):

    input = Input(shape=(224, 224, 3))

    x_l_0 = Patches(patch_size=32)(input)
    x_g_0 = input

    x_l = Local_Net(x_l_0, eps)
    x_g = Global_Net(x_g_0, eps)

    x_l = GlobalAveragePooling2D()(x_l)
    x_g = GlobalAveragePooling2D()(x_g)

    x_c = Concatenate()([x_l, x_g])

    x_l = Dense(units=1024, activation=relu)(x_l)
    x_g = Dense(units=1024, activation=relu)(x_g)

    share_1 = Dense(units=2048, activation=relu)
    share_2 = Dense(units=2048, activation=relu)
    share_3  = Dense(units=7, activation=softmax)

    x_l = share_1(x_l)
    x_g = share_1(x_g)
    x_c = share_1(x_c)

    x_l = share_2(x_l)
    x_g = share_2(x_g)
    x_c = share_2(x_c)

    out_l = share_3(x_l)
    out_g = share_3(x_g)
    out_c = share_3(x_c)

    model = Model(input, [out_l, out_g, out_c])

    return model

model = Enhance_Net()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patches (Patches)   │ (None, 32, 32,    │          0 │ input_layer[0][0] │
│                     │ 147)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 32, 32,    │    922,112 │ patches[0][0]     │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 224, 224,  │      9,472 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 32, 32,    │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 224, 224,  │        256 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 224, 224,  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 32, 32,    │    147,584 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 224, 224,  │     36,928 │ max_pooling2d_5[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        512 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 224, 224,  │        256 │ conv2d_6[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 16, 16,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 112, 112,  │     73,856 │ max_pooling2d_6[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │      1,024 │ conv2d_2[0][0]  

 Total params: 13,871,175 (52.91 MB)

 Trainable params: 13,866,055 (52.89 MB)

 Non-trainable params: 5,120 (20.00 KB)

In [7]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import os
import cv2
import h5py
import skimage

from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import load_model, Model

import os

output_dir = './outputs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)


def Add_Blur(sample, k_size):
    sample_blur = []
    for i in range(len(sample)):
        sample_blur.append(cv2.GaussianBlur(sample[i], ksize=(k_size, k_size), sigmaX=0))

    return np.array(sample_blur)

def Add_Noise(sample):
    sample_noise = []
    for i in range(len(sample)):
        np.random.seed(42)
        noisy_img = skimage.util.random_noise(sample[i], mode='gaussian', clip=True)

        sample_noise.append(noisy_img)

    return np.array(sample_noise)

if __name__ == '__main__':

    x_te_1, x_tr_1, y_te_1, y_tr_1 = x[:65], x[65:], y[:65], y[65:]
    x_te_2, x_tr_2, y_te_2, y_tr_2 = x[65:130], np.concatenate((x[:65], x[130:]), axis=0), y[65:130], np.concatenate((y[:65], y[130:]), axis=0)
    x_te_3, x_tr_3, y_te_3, y_tr_3 = x[130:195], np.concatenate((x[:130], x[195:]), axis=0), y[130:195], np.concatenate((y[:130], y[195:]), axis=0)
    x_te_4, x_tr_4, y_te_4, y_tr_4 = x[195:260], np.concatenate((x[:195], x[260:]), axis=0), y[195:260], np.concatenate((y[:195], y[260:]), axis=0)
    x_te_5, x_tr_5, y_te_5, y_tr_5 = x[260:], x[:260], y[260:], y[:260]

    # model = Enhance_Net()
    opt = keras.optimizers.Nadam(learning_rate=0.0001)
    model.compile(optimizer=opt,
                  loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'],
                  metrics=['accuracy', 'accuracy', 'accuracy'])

    for i in range(5):
        if i == 0:
            # x_te, x_tr = x_te_1, x_tr_1
            # x_te, x_tr = Add_Blur(sample=x_te_1, k_size=15), Add_Blur(sample=x_tr_1, k_size=15)
            x_te, x_tr = Add_Noise(sample=x_te_1), Add_Noise(sample=x_tr_1)
            y_te, y_tr = to_categorical(y_te_1, 7), to_categorical(y_tr_1, 7)
        elif i == 1:
            # x_te, x_tr = x_te_2, x_tr_2
            # x_te, x_tr = Add_Blur(sample=x_te_2, k_size=15), Add_Blur(sample=x_tr_2, k_size=15)
            x_te, x_tr = Add_Noise(sample=x_te_2), Add_Noise(sample=x_tr_2)
            y_te, y_tr = to_categorical(y_te_2, 7), to_categorical(y_tr_2, 7)
        elif i == 2:
            # x_te, x_tr = x_te_3, x_tr_3
            # x_te, x_tr = Add_Blur(sample=x_te_3, k_size=15), Add_Blur(sample=x_tr_3, k_size=15)
            x_te, x_tr = Add_Noise(sample=x_te_3), Add_Noise(sample=x_tr_3)
            y_te, y_tr = to_categorical(y_te_3, 7), to_categorical(y_tr_3, 7)
        elif i == 3:
            # x_te, x_tr = x_te_4, x_tr_4
            # x_te, x_tr = Add_Blur(sample=x_te_4, k_size=15), Add_Blur(sample=x_tr_4, k_size=15)
            x_te, x_tr = Add_Noise(sample=x_te_4), Add_Noise(sample=x_tr_4)
            y_te, y_tr = to_categorical(y_te_4, 7), to_categorical(y_tr_4, 7)
        else:
            # x_te, x_tr = x_te_5, x_tr_5
            # x_te, x_tr = Add_Blur(sample=x_te_5, k_size=15), Add_Blur(sample=x_tr_5, k_size=15)
            x_te, x_tr = Add_Noise(sample=x_te_5), Add_Noise(sample=x_tr_5)
            y_te, y_tr = to_categorical(y_te_5, 7), to_categorical(y_tr_5, 7)

        print(x_te.shape, x_tr.shape, y_te.shape, y_tr.shape)

        print('############# The {} Fold #############'.format(i+1))

        file_path = './weights/best_weights_ck_noise_{}.weights.h5'.format(i + 1)

        reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                      factor=0.1,
                                      patience=200,
                                      verbose=1,
                                      mode='auto',
                                      min_delta=0.0001,
                                      min_lr=0)
        checkpoint = ModelCheckpoint(filepath=file_path,
                                     monitor='val_loss',
                                     mode='auto',
                                     save_best_only=True,
                                     verbose=1,
                                     save_weights_only=True
                                     )

        model.fit(x=x_tr, y=[y_tr, y_tr, y_tr],
                  batch_size=32,
                  epochs=20,
                  verbose=1,
                  callbacks=[reduce_lr, checkpoint],
                  validation_split=0.2)

        model.load_weights('./weights/best_weights_ck_noise_{}.weights.h5'.format(i + 1))
        model_pred = Model(inputs=model.input, outputs=[model.get_layer('dense').output, model.get_layer('dense_1').output, model.get_layer('concatenate').output])
        x_l_tr_ck, x_g_tr_ck, x_c_tr_ck = model_pred.predict(x=x_tr, batch_size=32, verbose=1)
        model_pred = Model(inputs=model.input, outputs=[model.get_layer('dense').output, model.get_layer('dense_1').output, model.get_layer('concatenate').output])
        x_l_te_ck, x_g_te_ck, x_c_te_ck = model_pred.predict(x=x_te, batch_size=32, verbose=1)

        with h5py.File('./outputs/ck_sample_label_noise_{}.h5'.format(i + 1), 'w') as f:
            f.create_dataset(name='x_l_tr_ck', data=x_l_tr_ck)
            f.create_dataset(name='x_g_tr_ck', data=x_g_tr_ck)
            f.create_dataset(name='x_c_tr_ck', data=x_c_tr_ck)
            f.create_dataset(name='x_l_te_ck', data=x_l_te_ck)
            f.create_dataset(name='x_g_te_ck', data=x_g_te_ck)
            f.create_dataset(name='x_c_te_ck', data=x_c_te_ck)
            f.create_dataset(name='label_tr', data=y_tr)
            f.create_dataset(name='label_te', data=y_te)

Physical devices cannot be modified after being initialized
(65, 224, 224, 3) (855, 224, 224, 3) (65, 7) (855, 7)
############# The 1 Fold #############
Epoch 1/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 628ms/step - dense_4_accuracy: 0.5965 - dense_4_accuracy_1: 0.5965 - dense_4_accuracy_2: 0.5763 - dense_4_loss: 1.5915 - loss: 4.3696
Epoch 1: val_loss improved from inf to 5.47491, saving model to ./weights/best_weights_ck_noise_1.weights.h5
22/22 ━━━━━━━━━━━━━━━━━━━━ 48s 959ms/step - dense_4_accuracy: 0.6004 - dense_4_accuracy_1: 0.6004 - dense_4_accuracy_2: 0.5802 - dense_4_loss: 1.5779 - loss: 4.3384 - val_dense_4_accuracy: 0.7953 - val_dense_4_accuracy_1: 0.7953 - val_dense_4_accuracy_2: 0.7544 - val_dense_4_loss: 1.8399 - val_loss: 5.4749 - learning_rate: 1.0000e-04
Epoch 2/20
21/22 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - dense_4_accuracy: 0.6929 - dense_4_accuracy_1: 0.6926 - dense_4_accuracy_2: 0.6901 - dense_4_loss: 1.0967 - loss: 3.2730
Epoch 2: val_loss improved from 5.47491 to 5.02457, s

27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 751ms/step
(65, 224, 224, 3) (855, 224, 224, 3) (65, 7) (855, 7)
############# The 3 Fold #############
Epoch 1/20
21/22 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - dense_4_accuracy: 0.8127 - dense_4_accuracy_1: 0.7918 - dense_4_accuracy_2: 0.8560 - dense_4_loss: 0.3862 - loss: 1.4719
Epoch 1: val_loss improved from inf to 2.89469, saving model to ./weights/best_weights_ck_noise_3.weights.h5
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - dense_4_accuracy: 0.8135 - dense_4_accuracy_1: 0.7921 - dense_4_accuracy_2: 0.8560 - dense_4_loss: 0.3867 - loss: 1.4733 - val_dense_4_accuracy: 0.6667 - val_dense_4_accuracy_1: 0.7953 - val_dense_4_accuracy_2: 0.8012 - val_dense_4_loss: 0.8090 - val_loss: 2.8947 - learning_rate: 1.0000e-04
Epoch 2/20
21/22 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - dense_4_accuracy: 0.8647 - dense_4_accuracy_1: 0.8230 - dense_4_accuracy_2: 0.8948 - dense_4_loss: 0.2831 - loss: 1.1573
Epoch 2: val_loss did not improve fro

27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 760ms/step
(65, 224, 224, 3) (855, 224, 224, 3) (65, 7) (855, 7)
############# The 4 Fold #############
Epoch 1/20
21/22 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - dense_4_accuracy: 0.8194 - dense_4_accuracy_1: 0.8066 - dense_4_accuracy_2: 0.8432 - dense_4_loss: 0.3616 - loss: 1.3535
Epoch 1: val_loss improved from inf to 2.96220, saving model to ./weights/best_weights_ck_noise_4.weights.h5
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - dense_4_accuracy: 0.8192 - dense_4_accuracy_1: 0.8074 - dense_4_accuracy_2: 0.8439 - dense_4_loss: 0.3619 - loss: 1.3567 - val_dense_4_accuracy: 0.7544 - val_dense_4_accuracy_1: 0.7953 - val_dense_4_accuracy_2: 0.7953 - val_dense_4_loss: 0.9553 - val_loss: 2.9622 - learning_rate: 1.0000e-04
Epoch 2/20
21/22 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - dense_4_accuracy: 0.8591 - dense_4_accuracy_1: 0.8423 - dense_4_accuracy_2: 0.8943 - dense_4_loss: 0.2655 - loss: 1.0941
Epoch 2: val_loss did not improve fro

In [ ]:
from huggingface_hub import login
from huggingface_hub import upload_file
from google.colab import userdata

token_hug = userdata.get('token_hugging')

login(token=token_hug)

for i in range(1,6):
  upload_file(
      path_or_fileobj=f"/content/weights/best_weights_ck_noise_{i}.weights.h5",   # file local
      path_in_repo=f"weights/best_weights_ck_noise_{i}.weights.h5",                    # tên file khi upload lên repo
      repo_id="DaoHuyen/XLA",                      # repo đích
  )


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...hts_ck_noise_1.weights.h5:   2%|2         | 3.97MB /  167MB            

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

...

New Data Upload               : |          |  0.00B /  0.00B            

  ...hts_ck_noise_4.weights.h5:   2%|2         | 3.96MB /  167MB            

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...hts_ck_noise_5.weights.h5:   2%|2         | 3.96MB /  167MB            

## Part 2


## fusion

### model

In [9]:
import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow.keras.layers import Input, Dense, Lambda, Multiply, Add, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu, softmax

def Cosin_similarity(input):
    dot1 = K.batch_dot(input[0], input[1], axes=1)
    dot2 = K.batch_dot(input[0], input[0], axes=1)
    dot3 = K.batch_dot(input[1], input[1], axes=1)
    max = K.maximum(K.sqrt(dot2 * dot3), K.epsilon())
    value = dot1 / max
    return K.tanh(value)

def Bund(input):
    alpha_1 = input[0]
    alpha_2 = input[1]

    alpha_l = alpha_1/(alpha_1+alpha_2)
    alpha_g = alpha_2/(alpha_1+alpha_2)

    return alpha_l, alpha_g

def Fusion_Net():
    input_1 = Input(shape=(1024, ))
    input_2 = Input(shape=(1024, ))
    input_3 = Input(shape=(1024, ))

    x_l = Dense(units=2048, activation=relu)(input_1)
    x_g = Dense(units=2048, activation=relu)(input_2)
    x_c = Dense(units=2048, activation=relu)(input_3)

    alpha_1 = Lambda(Cosin_similarity)([x_l, x_c])
    alpha_2 = Lambda(Cosin_similarity)([x_g, x_c])
    alpha_l, alpha_g = Lambda(Bund)([alpha_1, alpha_2])
    out_l = Multiply()([alpha_l, x_l])
    out_g = Multiply()([alpha_g, x_g])

    out = Concatenate()([out_l, out_g])

    out = Dense(units=7, activation=softmax)(out)

    model = Model([input_1, input_2, input_3], out)

    return model

### train

In [13]:
import h5py
import tensorflow as tf
import numpy as np
import os

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow import keras
from sklearn.metrics import balanced_accuracy_score, accuracy_score

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

if __name__ == '__main__':

    ACC = []

    model = Fusion_Net()
    model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['acc'])

    for i in range(5):
        model = Fusion_Net()
        model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['acc'])
        with h5py.File('./outputs/ck_sample_label_noise_{}.h5'.format(i+1), 'r') as f:
            x_l_tr_ck = np.array(f['x_l_tr_ck'])
            x_g_tr_ck = np.array(f['x_g_tr_ck'])
            x_c_tr_ck = np.array(f['x_c_tr_ck'])
            x_l_te_ck = np.array(f['x_l_te_ck'])
            x_g_te_ck = np.array(f['x_g_te_ck'])
            x_c_te_ck = np.array(f['x_c_te_ck'])
            label_tr = np.array(f['label_tr'])
            label_te = np.array(f['label_te'])

        # label_tr, label_te = to_categorical(label_tr), to_categorical(label_te)
        print(x_l_tr_ck.shape, x_g_tr_ck.shape, x_c_tr_ck.shape, x_l_te_ck.shape, x_g_te_ck.shape, x_c_te_ck.shape, label_tr.shape, label_te.shape)

        file_path = './models/CK_best_model_noise_{}.h5'.format(i + 1)

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc',
                                                      verbose=1,
                                                      mode='auto',
                                                      factor=0.1,
                                                      min_delta=0.001,
                                                      patience=200,
                                                      cooldown=0,
                                                      min_lr=0.00000001)

        checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path,
                                                     monitor='val_acc',
                                                     mode='auto',
                                                     save_best_only=True,
                                                     verbose=1,
                                                     )

        model.fit(x=[x_l_tr_ck, x_g_tr_ck, x_c_tr_ck], y=label_tr,
                  batch_size=32,
                  epochs=30,
                  verbose=2,
                  callbacks=[checkpoint, reduce_lr],
                  validation_split=0.2)

        model = load_model(
            filepath=file_path,
            custom_objects={
                "Cosin_similarity": Cosin_similarity,
                "Bund": Bund
            }
        )


        y_pred = model.predict([x_l_te_ck, x_g_te_ck, x_c_te_ck], batch_size=32, verbose=1)
        y_pred = np.argmax(y_pred, axis=1)
        y_true = np.argmax(label_te, axis=1)

        ACC.append(accuracy_score(y_true=y_true, y_pred=y_pred))

    print(ACC)
    print(np.mean(ACC))

Physical devices cannot be modified after being initialized
(855, 1024) (855, 1024) (855, 1024) (65, 1024) (65, 1024) (65, 1024) (855, 7) (65, 7)
Epoch 1/30

Epoch 1: val_acc improved from -inf to 0.80702, saving model to ./models/CK_best_model_noise_1.h5


22/22 - 6s - 274ms/step - acc: 0.6827 - loss: 1.1384 - val_acc: 0.8070 - val_loss: 0.8241 - learning_rate: 1.0000e-04
Epoch 2/30

Epoch 2: val_acc did not improve from 0.80702
22/22 - 0s - 7ms/step - acc: 0.7573 - loss: 0.8175 - val_acc: 0.8070 - val_loss: 0.7812 - learning_rate: 1.0000e-04
Epoch 3/30

Epoch 3: val_acc did not improve from 0.80702
22/22 - 0s - 7ms/step - acc: 0.7836 - loss: 0.6809 - val_acc: 0.8070 - val_loss: 0.8057 - learning_rate: 1.0000e-04
Epoch 4/30

Epoch 4: val_acc did not improve from 0.80702
22/22 - 0s - 7ms/step - acc: 0.7924 - loss: 0.6033 - val_acc: 0.8012 - val_loss: 0.7991 - learning_rate: 1.0000e-04
Epoch 5/30

Epoch 5: val_acc did not improve from 0.80702
22/22 - 0s - 7ms/step - acc: 0.8026 - loss: 0.5457 - val_acc: 0.7895 - val_loss: 0.8309 - learning_rate: 1.0000e-04
Epoch 6/30

Epoch 6: val_acc did not improve from 0.80702
22/22 - 0s - 7ms/step - acc: 0.8143 - loss: 0.5005 - val_acc: 0.7953 - val_loss: 0.8269 - learning_rate: 1.0000e-04
Epoch 7/30



22/22 - 0s - 15ms/step - acc: 0.8860 - loss: 0.3386 - val_acc: 0.8129 - val_loss: 0.9099 - learning_rate: 1.0000e-04
Epoch 14/30

Epoch 14: val_acc did not improve from 0.81287
22/22 - 0s - 7ms/step - acc: 0.8889 - loss: 0.3198 - val_acc: 0.8129 - val_loss: 0.9253 - learning_rate: 1.0000e-04
Epoch 15/30

Epoch 15: val_acc did not improve from 0.81287
22/22 - 0s - 7ms/step - acc: 0.9020 - loss: 0.3049 - val_acc: 0.8129 - val_loss: 0.9330 - learning_rate: 1.0000e-04
Epoch 16/30

Epoch 16: val_acc did not improve from 0.81287
22/22 - 0s - 7ms/step - acc: 0.9020 - loss: 0.2917 - val_acc: 0.8070 - val_loss: 0.9605 - learning_rate: 1.0000e-04
Epoch 17/30

Epoch 17: val_acc did not improve from 0.81287
22/22 - 0s - 7ms/step - acc: 0.9079 - loss: 0.2831 - val_acc: 0.8129 - val_loss: 0.9604 - learning_rate: 1.0000e-04
Epoch 18/30

Epoch 18: val_acc did not improve from 0.81287
22/22 - 0s - 7ms/step - acc: 0.9050 - loss: 0.2715 - val_acc: 0.8129 - val_loss: 0.9683 - learning_rate: 1.0000e-04
Epo

22/22 - 0s - 14ms/step - acc: 0.9532 - loss: 0.1845 - val_acc: 0.8187 - val_loss: 1.1072 - learning_rate: 1.0000e-04
Epoch 30/30

Epoch 30: val_acc did not improve from 0.81871
22/22 - 0s - 8ms/step - acc: 0.9488 - loss: 0.1776 - val_acc: 0.8012 - val_loss: 1.1188 - learning_rate: 1.0000e-04


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 281ms/step
(855, 1024) (855, 1024) (855, 1024) (65, 1024) (65, 1024) (65, 1024) (855, 7) (65, 7)
Epoch 1/30

Epoch 1: val_acc improved from -inf to 0.80117, saving model to ./models/CK_best_model_noise_2.h5


...

22/22 - 0s - 14ms/step - acc: 0.8889 - loss: 0.3379 - val_acc: 0.8304 - val_loss: 0.7416 - learning_rate: 1.0000e-04
Epoch 7/30

Epoch 7: val_acc did not improve from 0.83041
22/22 - 0s - 7ms/step - acc: 0.8904 - loss: 0.3035 - val_acc: 0.8070 - val_loss: 0.7829 - learning_rate: 1.0000e-04
Epoch 8/30

Epoch 8: val_acc did not improve from 0.83041
22/22 - 0s - 7ms/step - acc: 0.9079 - loss: 0.2767 - val_acc: 0.8129 - val_loss: 0.7813 - learning_rate: 1.0000e-04
Epoch 9/30

Epoch 9: val_acc did not improve from 0.83041
22/22 - 0s - 7ms/step - acc: 0.9137 - loss: 0.2541 - val_acc: 0.8304 - val_loss: 0.7915 - learning_rate: 1.0000e-04
Epoch 10/30

Epoch 10: val_acc did not improve from 0.83041
22/22 - 0s - 7ms/step - acc: 0.9108 - loss: 0.2469 - val_acc: 0.8187 - val_loss: 0.7953 - learning_rate: 1.0000e-04
Epoch 11/30

Epoch 11: val_acc did not improve from 0.83041
22/22 - 0s - 7ms/step - acc: 0.9313 - loss: 0.2179 - val_acc: 0.8129 - val_loss: 0.8047 - learning_rate: 1.0000e-04
Epoch 12/

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 269ms/step
(260, 1024) (260, 1024) (260, 1024) (660, 1024) (660, 1024) (660, 1024) (260, 7) (660, 7)
Epoch 1/30

Epoch 1: val_acc improved from -inf to 0.69231, saving model to ./models/CK_best_model_noise_5.h5


7/7 - 7s - 968ms/step - acc: 0.6346 - loss: 1.2309 - val_acc: 0.6923 - val_loss: 1.1062 - learning_rate: 1.0000e-04
Epoch 2/30

Epoch 2: val_acc did not improve from 0.69231
7/7 - 0s - 17ms/step - acc: 0.7356 - loss: 0.8503 - val_acc: 0.6923 - val_loss: 1.0767 - learning_rate: 1.0000e-04
Epoch 3/30

Epoch 3: val_acc improved from 0.69231 to 0.73077, saving model to ./models/CK_best_model_noise_5.h5


7/7 - 0s - 38ms/step - acc: 0.8029 - loss: 0.6597 - val_acc: 0.7308 - val_loss: 1.0248 - learning_rate: 1.0000e-04
Epoch 4/30

Epoch 4: val_acc did not improve from 0.73077
7/7 - 0s - 15ms/step - acc: 0.8365 - loss: 0.5266 - val_acc: 0.7308 - val_loss: 1.0023 - learning_rate: 1.0000e-04
Epoch 5/30

Epoch 5: val_acc did not improve from 0.73077
7/7 - 0s - 15ms/step - acc: 0.8558 - loss: 0.4312 - val_acc: 0.7308 - val_loss: 0.9909 - learning_rate: 1.0000e-04
Epoch 6/30

Epoch 6: val_acc did not improve from 0.73077
7/7 - 0s - 16ms/step - acc: 0.8750 - loss: 0.3558 - val_acc: 0.7308 - val_loss: 0.9931 - learning_rate: 1.0000e-04
Epoch 7/30

Epoch 7: val_acc did not improve from 0.73077
7/7 - 0s - 16ms/step - acc: 0.8846 - loss: 0.3014 - val_acc: 0.7308 - val_loss: 0.9834 - learning_rate: 1.0000e-04
Epoch 8/30

Epoch 8: val_acc did not improve from 0.73077
7/7 - 0s - 16ms/step - acc: 0.9279 - loss: 0.2534 - val_acc: 0.7308 - val_loss: 0.9633 - learning_rate: 1.0000e-04
Epoch 9/30

Epoch 9:

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
[0.7384615384615385, 0.7846153846153846, 0.8153846153846154, 0.8, 0.7666666666666667]
0.7810256410256409


In [ ]:
from huggingface_hub import login
from huggingface_hub import upload_file

login(token=token_hug)


upload_file(
    path_or_fileobj=f"/content/models/CK_best_model_noise_1.h5",   # file local
    path_in_repo=f"weights/CK_best_model_noise_1.h5",                    # tên file khi upload lên repo
    repo_id="DaoHuyen/XLA",                      # repo đích
)


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  .../CK_best_model_noise_1.h5:   9%|8         | 6.48MB / 76.0MB            

CommitInfo(commit_url='https://huggingface.co/DaoHuyen/XLA/commit/56ed02b63ed067eba8298bac6e37d73505a741d3', commit_message='Upload weights/CK_best_model_noise_1.h5 with huggingface_hub', commit_description='', oid='56ed02b63ed067eba8298bac6e37d73505a741d3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/DaoHuyen/XLA', endpoint='https://huggingface.co', repo_type='model', repo_id='DaoHuyen/XLA'), pr_revision=None, pr_num=None)